### This notebook is intended to predict sql injection attacks

In [72]:

# import required packages

import glob
import time
import pandas as pd
# from xml.dom import minidom
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [73]:
import keras

In [74]:
import nltk
import pandas as pd
import os

### File_name: git_foospidy_payloads_fuzzing_code_db_sqli.txt
##### Comments :  
1. removed \n at the end of each line
2.  replaced %20 by space
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [75]:
# preprocess sql data to have same format for all files

def clean_sqli_data(data):
    
    for i in range(len(data)):
        
        data[i]=data[i].replace('\n', '')
        data[i]=data[i].replace('%20', ' ')
        data[i]=data[i].replace('=', ' = ')
        data[i]=data[i].replace('((', ' (( ')
        data[i]=data[i].replace('))', ' )) ')
        data[i]=data[i].replace('(', ' ( ')
        data[i]=data[i].replace(')', ' ) ')
    
    return data

In [76]:
path='./data/collection/git_foospidy_payloads_fuzzing_code_db_sqli.txt'

In [77]:
# read data from file

sql_lines_fuzzing=[]
f = open(path, "r")
for x in f:
    sql_lines_fuzzing.append(x)

In [78]:
sql_lines_fuzzing=clean_sqli_data(sql_lines_fuzzing) 

In [79]:
sql_lines_fuzzing[:15]

["'sqlvuln",
 "'+sqlvuln",
 'sqlvuln;',
 ' ( sqlvuln ) ',
 "a' or 1 = 1--",
 '"a"" or 1 = 1--"',
 ' or a  =  a',
 "a' or 'a'  =  'a",
 '1 or 1 = 1',
 "a' waitfor delay '0:0:10'--",
 "1 waitfor delay '0:0:10'--",
 'declare @q nvarchar  ( 4000 )  select @q  = ',
 '0x770061006900740066006F0072002000640065006C00610079002000270030003A0030003A',
 '0',
 '031003000270000']

### File_Name: git_foospidy_payloads_other_sqli_camoufl4g3.txt
##### Comments : ok
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [80]:
path='./data/collection/git_foospidy_payloads_other_sqli_camoufl4g3.txt'

In [81]:
# read data from file

sql_lines_camoufl4g3=[]
f = open(path, "r")
for x in f:
    sql_lines_camoufl4g3.append(x)

In [82]:
sql_lines_camoufl4g3[:15] # data before cleaning

["'-'\n",
 "' '\n",
 "'&'\n",
 "'^'\n",
 "'*'\n",
 "' or ''-'\n",
 "' or '' '\n",
 "' or ''&'\n",
 "' or ''^'\n",
 "' or ''*'\n",
 '"-"\n',
 '" "\n',
 '"&"\n',
 '"^"\n',
 '"*"\n']

In [83]:
sql_lines_camoufl4g3=clean_sqli_data(sql_lines_camoufl4g3)

In [84]:
sql_lines_camoufl4g3[:15]  # data after cleaning

["'-'",
 "' '",
 "'&'",
 "'^'",
 "'*'",
 "' or ''-'",
 "' or '' '",
 "' or ''&'",
 "' or ''^'",
 "' or ''*'",
 '"-"',
 '" "',
 '"&"',
 '"^"',
 '"*"']

### File_Name: git_foospidy_payloads_other_sqli_libinjection-bypasses.txt
##### Comments: Should Remove &()o1: from start of each sentence
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [85]:
path='./data/collection/git_foospidy_payloads_other_sqli_libinjection-bypasses.txt'

In [86]:
# read data from file

sql_lines_bypasses=[]
f = open(path, "r")
for x in f:
    sql_lines_bypasses.append(x)

In [87]:
sql_lines_bypasses[:5] # before cleaning

['&()o1: select * from users where id=1 or (\\)=1 union select 1,@@VERSION -- 1\n',
 '&(.)o: select * from users where id=1 or (\\.)=1 union select 1,@@VERSION -- 1\n',
 '&(1&1: select * from users where id=1 or (\\+)=1 or 1=1 -- 1\n',
 '&(1)o: select * from users where id=1 or (1)=1 union select 1,banner from v$version where rownum=1 -- 1\n',
 '&(1UE: select * from users where id=1 or (\\+)=1 union select 1,@@VERSION -- 1\n']

In [88]:
sql_lines_bypasses=clean_sqli_data(sql_lines_bypasses)

In [89]:
sql_lines_bypasses[:5]  # data after cleaning

['& (  ) o1: select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1',
 '& ( . ) o: select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1',
 '& ( 1&1: select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1',
 '& ( 1 ) o: select * from users where id = 1 or  ( 1 )  = 1 union select 1,banner from v$version where rownum = 1 -- 1',
 '& ( 1UE: select * from users where id = 1 or  ( \\+ )  = 1 union select 1,@@VERSION -- 1']

In [90]:
# if don't want &(*)* sign in beginning of each sentence then run next code else don't

for i in range(len(sql_lines_bypasses)):
    sentence=sql_lines_bypasses[i]
    sql_lines_bypasses[i]=sentence.split(':')[1]
    

In [91]:
sql_lines_bypasses[:5]

[' select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1',
 ' select * from users where id = 1 or  ( 1 )  = 1 union select 1,banner from v$version where rownum = 1 -- 1',
 ' select * from users where id = 1 or  ( \\+ )  = 1 union select 1,@@VERSION -- 1']

### File_Name: git_foospidy_payloads_owasp_sqli.txt
##### Comments: ok
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [92]:
path='./data/collection/git_foospidy_payloads_owasp_sqli.txt'

In [93]:
# read data from file

sql_lines_owasp=[]
f = open(path, "r")
for x in f:
    sql_lines_owasp.append(x)

In [94]:
sql_lines_owasp[0:15] # before cleaning

['a\n',
 "a' \n",
 "a' --\n",
 "a' or 1=1; --\n",
 '@\n',
 '?\n',
 "' and 1=0) union all\n",
 '? or 1=1 --\n',
 "x' and userid is NULL; --\n",
 "x' and email is NULL; --\n",
 "anything' or 'x'='x\n",
 "x' and 1=(select count(*) from tabname); --\n",
 "x' and members.email is NULL; --\n",
 "x' or full_name like '%bob%\n",
 '23 or 1=1; --\n']

In [95]:
sql_lines_owasp=clean_sqli_data(sql_lines_owasp)

In [96]:
sql_lines_owasp[:15] # after cleaning

['a',
 "a' ",
 "a' --",
 "a' or 1 = 1; --",
 '@',
 '?',
 "' and 1 = 0 )  union all",
 '? or 1 = 1 --',
 "x' and userid is NULL; --",
 "x' and email is NULL; --",
 "anything' or 'x' = 'x",
 "x' and 1 =  ( select count ( * )  from tabname ) ; --",
 "x' and members.email is NULL; --",
 "x' or full_name like '%bob%",
 '23 or 1 = 1; --']

### File_Name: git_seclist_fuzzing_sqli_Generic-SQLi.txt
##### comments: should we remove %20 by spaces because %20 equals to space
#### Source: https://github.com/danielmiessler/SecLists/tree/master/Fuzzing/SQLi

In [97]:
path='./data/collection/git_seclist_fuzzing_sqli_Generic-SQLi.txt'

In [98]:
# read data from file

sql_lines_Generic=[]
f = open(path, "r")
for x in f:
    sql_lines_Generic.append(x)

In [99]:
sql_lines_Generic[:15] # before cleaning

[")%20or%20('x'='x\n",
 '%20or%201=1\n',
 "; execute immediate 'sel' || 'ect us' || 'er'\n",
 'benchmark(10000000,MD5(1))#\n',
 'update\n',
 '";waitfor delay \'0:0:__TIME__\'--\n',
 '1) or pg_sleep(__TIME__)--\n',
 '||(elt(-3+5,bin(15),ord(10),hex(char(45))))\n',
 '"hi"") or (""a""=""a"\n',
 'delete\n',
 'like\n',
 '" or sleep(__TIME__)#\n',
 'pg_sleep(__TIME__)--\n',
 '*(|(objectclass=*))\n',
 'declare @q nvarchar (200) 0x730065006c00650063 ...\n']

In [100]:
sql_lines_Generic=clean_sqli_data(sql_lines_Generic)

In [101]:
sql_lines_Generic[:15] # after cleaning

[" )  or  ( 'x' = 'x",
 ' or 1 = 1',
 "; execute immediate 'sel' || 'ect us' || 'er'",
 'benchmark ( 10000000,MD5 ( 1  )  )  #',
 'update',
 '";waitfor delay \'0:0:__TIME__\'--',
 '1 )  or pg_sleep ( __TIME__ ) --',
 '|| ( elt ( -3+5,bin ( 15 ) ,ord ( 10 ) ,hex ( char ( 45  )  )    )  )  ',
 '"hi"" )  or  ( ""a"" = ""a"',
 'delete',
 'like',
 '" or sleep ( __TIME__ ) #',
 'pg_sleep ( __TIME__ ) --',
 '* ( | ( objectclass = *  )  )  ',
 'declare @q nvarchar  ( 200 )  0x730065006c00650063 ...']

### File_name: scottparker_ml_sqli_src__trainingdata_plain.txt
##### Comments: 
removed last 22 records containing URLs
#### Source: https://github.com/tungpv98/Detect-Sql-Injection-by-Machine-Learning/tree/ebeb3287931677a3a42f11a7a08dbc13e374ff05/Sql-Injection/source/trainingdata

In [102]:

# function to remove stopwords

stop_words = set(stopwords.words('english')) 

def fun_remove_stop_words(posts):

    filtered=''
    
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    
    return filtered

In [103]:
path='./data/collection/'
file="scottparker_ml_sqli_src__trainingdata_plain.txt"

In [104]:
#read benign data

df = pd.read_csv(os.path.join(path,file), sep='Aw3s0meSc0t7', names=['benign'], header=None, engine='python')

In [105]:
df.head()

,benign
0,Add plain text here
1,Ne te quaesiveris extra.
2,Man is his own star; and the soul that can
3,"Render an honest and a perfect man,"
4,"Commands all light, all influence, all fate;"


In [106]:
plain_text=df['benign'].values  # get sentences

In [107]:
plain_text[:5]

array(['Add plain text here', 'Ne te quaesiveris extra.',
       'Man is his own star; and the soul that can',
       'Render an honest and a perfect man,',
       'Commands all light, all influence, all fate;'], dtype=object)

In [108]:
plain_text=plain_text[:-22] # removed last 22 records that were urls

In [109]:
len(plain_text)

3672

In [110]:
# convert from list to string

data=''
for x in plain_text:
    data+=" " + x

In [111]:
type(data)

str

In [112]:
data=fun_remove_stop_words(data)  # remove stop words
data=data.split('.')              # split sentences

In [113]:
# seperate words inside tags

for i in range(len(data)):
    data[i]=data[i].replace('<', ' <')
    data[i]=data[i].replace('>', '> ')
    data[i]=data[i].replace('=', ' = ')

In [114]:
data[:5]

['  Add plain text Ne te quaesiveris extra',
 ' Man star; soul Render honest perfect man, Commands light, influence, fate; Nothing falls early late',
 ' Our acts angels are, good ill, Our fatal shadows walk us still',
 ' Epilogue Beaumont Fletcher’s Honest Man’s Fortune Cast bantling rocks, Suckle she-wolf’s teat; Wintered hawk fox, Power speed hands feet',
 ' I read day verses written eminent painter original conventional']

Statistics

In [115]:
print("Benign records: %2i" %len(data))

Benign records: 3072


In [116]:
# read self created benign data

path='./data/collection/benign_for_training.txt'
benign_data=[]
f = open(path, "r")
for x in f:
    benign_data.append(x)


In [117]:
len(benign_data)

476

In [118]:
print(f"SQL fuzzing : {len(sql_lines_fuzzing)}  camoufl4gs : {len(sql_lines_camoufl4g3)} parsed : {len(sql_lines_bypasses)} owasp : {len(sql_lines_owasp)} generic : {len(sql_lines_Generic)} \n total sql injection data : {len(sql_lines_Generic)+len(sql_lines_owasp)+len(sql_lines_bypasses)+len(sql_lines_camoufl4g3)+len(sql_lines_fuzzing)} ")

SQL fuzzing : 142  camoufl4gs : 77 parsed : 474 owasp : 167 generic : 268 
 total sql injection data : 1128 


#### combination of all sqli attacks

In [119]:
all_sqli_sentence=sql_lines_owasp+sql_lines_bypasses+sql_lines_camoufl4g3+sql_lines_fuzzing+sql_lines_Generic

In [120]:
len(all_sqli_sentence)

1128

In [121]:
# replace numeric values by a keyword 'numeric'
def optional_numeric_to_numeric(all_sqli_sentence):
    
    for i in range(len(all_sqli_sentence)):
        
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1 ', 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 1', 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("'1 ", "'numeric ")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 1'", " numeric'")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1,', 'numeric,')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("1\ ", 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("‘1", '‘numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 2 ", " numeric ")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 3 ', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 3--', ' numeric--')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 4 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 5 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 6 ', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 7 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 8 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1234', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("22", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 8 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 200 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("23 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('"1', '"numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1"', '"numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("7659", 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 37 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 45 ", ' numeric ')
    
    return all_sqli_sentence


In [122]:
all_sqli_sentence

['a',
 "a' ",
 "a' --",
 "a' or 1 = 1; --",
 '@',
 '?',
 "' and 1 = 0 )  union all",
 '? or 1 = 1 --',
 "x' and userid is NULL; --",
 "x' and email is NULL; --",
 "anything' or 'x' = 'x",
 "x' and 1 =  ( select count ( * )  from tabname ) ; --",
 "x' and members.email is NULL; --",
 "x' or full_name like '%bob%",
 '23 or 1 = 1; --',
 "'; exec master..xp_cmdshell 'ping 172.10.1.255'--",
 'a',
 '1 or 1 = 1',
 "1' or '1' = '1",
 "1 and user_name (  )   =  'dbo'",
 '1',
 "1'1",
 '1 exec sp_  ( or exec xp_ ) ',
 '1 and 1 = 1',
 "1' and 1 =  ( select count ( * )  from tablenames ) ; --",
 '1',
 "1 and user_name (  )   =  'dbo'",
 "\\'; desc users; --",
 "1\\'1",
 "1' and non_existant_table  =  '1",
 "' or username is not NULL or username  =  '",
 "1 and ascii ( lower ( substring  (  (  select top 1 name from sysobjects where xtype = 'u' ) , 1, 1  )  )   )  > 116",
 "1 union all select 1,2,3,4,5,6,name from sysobjects where xtype  =  'u' --",
 '1 uni/**/on select all from where',
 '’ or ‘1’ =

#### Combination of benign an attack data

In [123]:
import pandas as pd

In [124]:
# give labels to sql data

values=[]
for i in all_sqli_sentence:
    values.append((i,1))

In [125]:
# give labels to benign data

for i in data:
    values.append((i,0))

In [129]:
len(all_sqli_sentence)+len(data)

4200

In [126]:
len(values)

4200

In [127]:
for i in benign_data:
    values.append((i,0))

In [128]:
len(values)

4676

In [131]:
values[1]

("a' ", 1)

In [132]:
# convert to dataframe

df=pd.DataFrame(values,columns=['Sentence','Label'])



In [133]:
df.head()

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1


### Save data as csv

In [134]:
df.to_csv('sqli.csv', index=False, encoding='utf-16')

In [135]:
df=pd.read_csv('sqli.csv',encoding='utf-16')

In [136]:
# vectorization of data

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()



In [137]:
posts

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [138]:
posts.shape

(4676, 4906)

In [140]:
transformed_posts=pd.DataFrame(posts)

In [141]:
df=pd.concat([df,transformed_posts],axis=1)

In [143]:
df.head()

,Sentence,Label,0,1,2,3,4,5,6,7,...,4896,4897,4898,4899,4900,4901,4902,4903,4904,4905
0,a,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,a',1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,a' --,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,a' or 1 = 1; --,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,@,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
X=df[df.columns[2:]]

In [145]:
X.shape

(4676, 4906)

In [146]:
y=df['Label']

In [147]:
from sklearn.model_selection import train_test_split

In [148]:
# split train test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [149]:
X.shape

(4676, 4906)

In [150]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [151]:
from sklearn.metrics import accuracy_score

In [152]:
y_pred=clf.predict(X_test)

In [153]:
accuracy_score(y_test, y_pred)

0.9006410256410257

In [ ]:
for i,j in zip(y_test,y_pred):
    print(i==j)

In [154]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [155]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
# model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
# model.add(Activation('softmax'))

In [156]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                98140     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              11264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 1025      
Total params: 114,735
Trainable params: 112,687
Non-trainable params: 2,048
____________________________________________

In [157]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=15)


Train on 3740 samples, validate on 936 samples
Epoch 1/10
3740/3740 [==============================] - 4s 985us/step - loss: 0.2285 - accuracy: 0.8952 - val_loss: 0.5872 - val_accuracy: 0.7415
Epoch 2/10
3740/3740 [==============================] - 2s 663us/step - loss: 0.1431 - accuracy: 0.9348 - val_loss: 0.3308 - val_accuracy: 0.8088
Epoch 3/10
3740/3740 [==============================] - 2s 575us/step - loss: 0.1322 - accuracy: 0.9390 - val_loss: 0.1477 - val_accuracy: 0.9380
Epoch 4/10
3740/3740 [==============================] - 2s 645us/step - loss: 0.1245 - accuracy: 0.9465 - val_loss: 0.1405 - val_accuracy: 0.9412
Epoch 5/10
3740/3740 [==============================] - 2s 590us/step - loss: 0.1086 - accuracy: 0.9500 - val_loss: 0.1431 - val_accuracy: 0.9402
Epoch 6/10
3740/3740 [==============================] - 3s 746us/step - loss: 0.1162 - accuracy: 0.9508 - val_loss: 0.1556 - val_accuracy: 0.9359
Epoch 7/10
3740/3740 [==============================] - 2s 643us/step - loss:

In [158]:
pred=model.predict(X_test)

In [159]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [97]:
accuracy_score(y_test,pred)

0.9726190476190476

In [ ]:
for i,j in zip(y_test,pred):
    print(i==j)

#### Saving Model

In [169]:
from keras.models import load_model
import pickle

model.save('my_model_additional_data.h5')
with open('vectorizer_additional_data', 'wb') as fin:
    pickle.dump(vectorizer, fin)


### Confusion Matrix
#### Assumption:
1. SQLI attack type is positive

In [160]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [161]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [162]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [163]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [164]:
accuracy,precision,recall=confusion_matrix(y_test,pred)

In [165]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 Accuracy : 0.938034188034188 
 Precision : 0.823943661971831 
 Recall : 0.9669421487603306


In [166]:

from sklearn.metrics import precision_score
precision_score(y_test, pred)

0.823943661971831

In [167]:
from sklearn.metrics import recall_score
recall_score(y_test, pred)


0.9669421487603306

### Classifying user data

     This cell is also in file user_Data_predict.py No need to run this cell

In [1]:

import keras
from keras.models import load_model
import pickle

mymodel = load_model('my_model_additional_data.h5')
myvectorizer = pickle.load(open("vectorizer_additional_data", 'rb'))




def clean_data(input_val):

    input_val=input_val.replace('\n', '')
    input_val=input_val.replace('%20', ' ')
    input_val=input_val.replace('=', ' = ')
    input_val=input_val.replace('((', ' (( ')
    input_val=input_val.replace('))', ' )) ')
    input_val=input_val.replace('(', ' ( ')
    input_val=input_val.replace(')', ' ) ')
    input_val=input_val.replace('1 ', 'numeric')
    input_val=input_val.replace(' 1', 'numeric')
    input_val=input_val.replace("'1 ", "'numeric ")
    input_val=input_val.replace(" 1'", " numeric'")
    input_val=input_val.replace('1,', 'numeric,')
    input_val=input_val.replace(" 2 ", " numeric ")
    input_val=input_val.replace(' 3 ', ' numeric ')
    input_val=input_val.replace(' 3--', ' numeric--')
    input_val=input_val.replace(" 4 ", ' numeric ')
    input_val=input_val.replace(" 5 ", ' numeric ')
    input_val=input_val.replace(' 6 ', ' numeric ')
    input_val=input_val.replace(" 7 ", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace('1234', ' numeric ')
    input_val=input_val.replace("22", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace(" 200 ", ' numeric ')
    input_val=input_val.replace("23 ", ' numeric ')
    input_val=input_val.replace('"1', '"numeric')
    input_val=input_val.replace('1"', '"numeric')
    input_val=input_val.replace("7659", 'numeric')
    input_val=input_val.replace(" 37 ", ' numeric ')
    input_val=input_val.replace(" 45 ", ' numeric ')

    return input_val








def predict_sqli_attack():
    
    repeat=True
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)

    
    if input_val== '0':
        repeat=False
    
    

    input_val=clean_data(input_val)
    input_val=[input_val]



    input_val=myvectorizer.transform(input_val).toarray()

    result=mymodel.predict(input_val)


    print(beautify)
    
    
    if repeat == True:
        
        if result>0.5:
            print("ALERT :::: This can be SQL injection")


        elif result<=0.5:
            print("It seems to be safe")
            
        print(beautify)
            
        predict_sqli_attack()
            
    elif repeat == False:
        print( " Good Bye ")

 



Using TensorFlow backend.


In [ ]:
# check for sql injection cheat sheet

predict_sqli_attack()